In [1]:
#pip install transformers datasets scikit-learn torch

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("datasets/cleanedDataSecondModel_merged.csv")
df = df[['quote', 'category_encoded']]
df.columns = ['text', 'label']

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


In [2]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch

class QuoteDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx])
            for key, val in self.encodings.items()
        } | {'labels': torch.tensor(self.labels[idx])}

train_dataset = QuoteDataset(train_encodings, train_labels)
test_dataset = QuoteDataset(test_encodings, test_labels)


In [5]:
# pip install transformers[torch]


In [ ]:
# pip install accelerate>=0.26.0

In [4]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

num_labels = len(set(df['label']))
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels) #roberta

# Training arguments
training_args = TrainingArguments(
    output_dir="./roberta_results", #ganti roberta
    num_train_epochs=3, #4 trllu berat
    per_device_train_batch_size=2, #8 trllu berat
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    save_strategy="no",
    logging_dir="./roberta_logs",
    logging_steps=10
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,4.747200,4.628444
2,4.611100,4.621861
3,4.574800,4.633987


TrainOutput(global_step=1200, training_loss=4.629390878677368, metrics={'train_runtime': 2461.1417, 'train_samples_per_second': 0.975, 'train_steps_per_second': 0.488, 'total_flos': 127162192046400.0, 'train_loss': 4.629390878677368, 'epoch': 3.0})

In [5]:
import numpy as np
from sklearn.metrics import classification_report

preds = trainer.predict(test_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)
print(classification_report(test_labels, pred_labels))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         6
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         4
          12       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         4
          21       0.00    

c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Marsha\College Apps\Python\environments\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [6]:
#predict
test_quote = "Success is not final, failure is not fatal"

inputs = tokenizer(test_quote, return_tensors="pt", truncation=True, padding=True, max_length=128)
model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

print(f"\nPredicted label for: \"{test_quote}\" → {predicted_label}")



Predicted label for: "Success is not final, failure is not fatal" → 62


62 -> kindness